Wytłumaczenie kodu:
Dropdown: Pozwala użytkownikowi wybierać lokalizacje.
Mapa: Używam biblioteki dash_leaflet do stworzenia interaktywnej mapy.
Callback: Funkcja w Dash, która aktualizuje pozycję markera i centrum mapy na podstawie wybranej lokalizacji.
Po uruchomieniu tego kodu, Dash utworzy lokalny serwer, na którym będzie dostępna aplikacja z mapą i listą rozwijaną. Użytkownik będzie mógł wybrać lokalizację z listy, a mapa automatycznie skupi się na wybranej lokalizacji.

OPIS tego notebboka zrobic


w dashu1 beda pokazywane ogolne wykresy historyczne i beda dynamicznie sie zminiec na podstawie wybranej lokalizacji, tez obliczone wartosci dla paneli slonecznych

w dashu 2 bedzie to samo tylko z predykcja

Plan działania dla Dashboardu w Dash:

Przygotowanie Środowiska:

Upewnij się, że masz zainstalowane wszystkie niezbędne pakiety, takie jak dash, dash-core-components, dash-html-components, pandas, i plotly.

Wczytanie Danych:

In [2]:
import pandas as pd

# Wczytywanie danych
df_weather_final = pd.read_csv("../2. Data/processed/final_df_processed.csv")


In [3]:
df_weather_final.head()

,Unnamed: 0,date,city_name,lat,lon,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,clouds_all,weather_main,estimated_irradiance,temp_modifier,humidity_modifier,daily_energy_production_kWh,year,month
0,0,1979-01-01,Baden_Baden,48.765640,8.228524,-10.442083,-16.532917,-16.18,3.02,1013.666667,81.916667,3.681667,100,Snow,0.5,1.0,0.95,1.9,1979,1
1,1,1979-01-01,Bozanska,54.473028,18.302179,-14.100000,-20.617500,-19.14,-9.46,1002.416667,82.000000,6.157917,100,Snow,0.5,1.0,0.95,1.9,1979,1
2,2,1979-01-01,Walcz_Drugi,53.257039,16.521203,-16.437083,-23.322500,-21.26,-9.57,1003.708333,82.416667,3.682083,100,Clouds,0.5,1.0,0.95,1.9,1979,1
3,3,1979-01-02,Baden_Baden,48.765640,8.228524,-8.856667,-14.874167,-15.82,-3.96,1024.250000,84.208333,4.524583,100,Snow,0.5,1.0,0.95,1.9,1979,1
4,4,1979-01-02,Bozanska,54.473028,18.302179,-9.037917,-15.179167,-14.14,-7.40,1004.791667,85.375000,4.449583,100,Clouds,0.5,1.0,0.95,1.9,1979,1


Budowa Layoutu Dashboardu:
Struktura dashboardu z elementami takimi jak input box do wyboru lokalizacji, wyświetlanie mapy i różnych statystyk.


ja bym chcial miec nastepujcae elementy na swoim dashbordzie:
1. input gdie bedzie mozna wpisywac miejscowosc, mamy trzy tylko wiec jak bedzie wpisywane cos innego to ma zwrocic "nie ma takiej miejscowosci w bazie". Nie chce rozwijanej listy. - DONE

2. wykres sredniej miesiecznej temperatury w danej lokalizacji 

3.  wykres srdeniego zachmurzenia dla danej lokalizacji - DONE

4. wykres srednia miesieczna wilgotonosci - DONE

5. wykres sredniej miesiecznej preedkosci wiatru dla danej lokalizacji - DONE

6. wykres rocznej produkcji energi przez lata dla danej lokalizacji, zmiana lat zeby byla jako suwak.- DONE

7. wykres sredniej miesiecznej produkcji ennergii dla danej lokalizacji.

8. Osobna tabela gdzie bedzie pokazana daily_energy_production_kWh

9. Mapa z lokalizacja - DONE

ten kod uzupelnic

In [22]:
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Załadowanie danych
df = pd.read_csv("../2. Data/processed/final_df_processed.csv")
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Inicjalizacja aplikacji Dash
app = dash.Dash(__name__)

# Layout aplikacji
app.layout = html.Div([
    dcc.Input(id='city-input', type='text', placeholder='Wpisz nazwę miejscowości'),
    html.Button('Sprawdź', id='submit-button', n_clicks=0),
    html.Div("Otwórz w przeglądarce: http://127.0.0.1:8050/"),
    html.Div("Dostępne miejscowości: Baden_Baden, Bozanska, Walcz_Drugi"),
    html.Div(id='error-message'),
    html.Div([
        dcc.Graph(id='monthly-weather-chart', style={'width': '48%', 'display': 'inline-block', 'margin-right': '2%'}),
        dcc.Graph(id='map-chart', style={'width': '48%', 'display': 'inline-block', 'margin-left': '2%'})
    ]),
    html.Div([
        dcc.Graph(id='annual-energy-chart'),
        dcc.RangeSlider(
            id='year-range-slider',
            min=df['year'].min(),
            max=df['year'].max(),
            value=[df['year'].min(), df['year'].max()],
            marks={str(year): str(year) for year in df['year'].unique()},
            step=None
        )
    ])
])

# Callback dla dynamicznej aktualizacji i mapy
@app.callback(
    [Output('error-message', 'children'),
     Output('city-input', 'style'),
     Output('annual-energy-chart', 'figure'),
     Output('monthly-weather-chart', 'figure'),
     Output('map-chart', 'figure')],
    [Input('submit-button', 'n_clicks'),
     Input('year-range-slider', 'value')],
    [State('city-input', 'value')]
)
def update_output(n_clicks, year_range, city):
    if city not in df['city_name'].unique():
        return 'Nie ma takiej miejscowości w bazie', {'border': '2px solid red'}, {}, {}, {}

    filtered_df = df[(df['city_name'] == city) & (df['year'] >= year_range[0]) & (df['year'] <= year_range[1])]
    annual_energy_fig = px.bar(filtered_df, x='year', y='daily_energy_production_kWh', title=f'Roczna produkcja energii przez lata dla {city}')
    # annual_energy_fig.update_xaxes(
    #    dtick=1,  # Ustawienie, aby każdy rok był pokazany
    #    tickmode='linear',  # Linearne pokazywanie roków
    #    range=[df['year'].min(), df['year'].max()]  # Zakres od minimalnego do maksymalnego roku
    #)

    weather_attributes = ['clouds_all', 'humidity', 'wind_speed']
    attribute_labels = ['Zachmurzenie (%)', 'Wilgotność (%)', 'Prędkość wiatru (m/s)']
    monthly_weather_fig = make_subplots(rows=3, cols=1, subplot_titles=attribute_labels, vertical_spacing=0.2)
    for i, attr in enumerate(weather_attributes, start=1):
        monthly_data = filtered_df.groupby('month')[attr].mean().reset_index()
        monthly_weather_fig.add_trace(go.Scatter(x=monthly_data['month'], y=monthly_data[attr], mode='lines+markers'), row=i, col=1)
         # Ustawienie etykiet miesięcy na osi X
        monthly_weather_fig.update_xaxes(title_text="Miesiące", tickvals=list(range(1, 13)), row=i, col=1)

    monthly_weather_fig.update_layout(height=600, showlegend=False, title_text="Średnie miesięczne warunki pogodowe", title_x=0)
    monthly_weather_fig.update_yaxes(rangemode='tozero')

    # Mapa z lokalizacją
    location_data = df[df['city_name'] == city].drop_duplicates(subset=['city_name'])
    map_fig = px.scatter_geo(location_data, lat='lat', lon='lon', hover_name='city_name',
                             projection='natural earth', title='Lokalizacja na mapie Europy',
                             size_max=20, color_discrete_sequence=["red"])
    map_fig.update_geos(
        visible=True, showcountries=True, countrycolor="Black",
        showcoastlines=True, coastlinecolor="Black",
        showland=True, landcolor="LightGreen",
        projection_scale=2
    )
    map_fig.update_layout(
        geo=dict(
            scope='europe', projection_type='orthographic', showframe=False,
            showcoastlines=True, landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)'
        )
    )

    return '', {'border': '1px solid green'}, annual_energy_fig, monthly_weather_fig, map_fig

if __name__ == '__main__':
    app.run_server(debug=True)


Rzeczywiście, wykresy, które pokazują miesięczną produkcję energii na poziomie 3000 kWh i roczną produkcję poniżej 1000 kWh, wydają się być sprzeczne i mogą wskazywać na błąd w danych lub w sposobie ich prezentacji. Spróbujmy zdiagnozować problem:

Błędne oznaczenie osi: Możliwe, że osie wykresów zostały błędnie oznaczone lub dane zostały błędnie zinterpretowane podczas tworzenia wykresów.
Sumowanie danych: Sprawdź, czy podczas agregowania danych na wykresie rocznym przypadkowo nie podsumowano danych inaczej niż planowano. Na przykład, zamiast sumować miesięczne dane na rok, możliwe, że wykonano inne operacje, takie jak uśrednianie.
Jednostki i skalowanie: Upewnij się, że jednostki na obu wykresach są takie same i że oba wykresy skalują dane w ten sam sposób.
Aby zbadać te problemy, warto sprawdzić proces przygotowania danych do obu wykresów. Oto przykładowy kod, który mógłby pomóc weryfikować, czy agregacja danych jest przeprowadzana poprawnie:

In [22]:
import pandas as pd

# Załaduj dane
df = pd.read_csv("../2. Data/processed/final_df_processed.csv")

# Przekonwertuj kolumnę z datami na typ datetime
df['date'] = pd.to_datetime(df['date'])

# Agreguj dane do miesięcznej produkcji
monthly_production = df.groupby(df['date'].dt.month)['daily_energy_production_kWh'].sum()

# Agreguj dane do rocznej produkcji
annual_production = df.groupby(df['date'].dt.year)['daily_energy_production_kWh'].sum()

print("Miesięczna produkcja energii (sumy miesięczne):")
print(monthly_production)

print("Roczna produkcja energii (sumy roczne):")
print(annual_production)


Miesięczna produkcja energii (sumy miesięczne):
date
1      8719.62
2      8389.50
3      9932.24
4     10316.10
5     10818.50
6     10211.80
7     10529.40
8     10575.70
9      9878.74
10     9574.26
11     8398.96
12     8642.98
Name: daily_energy_production_kWh, dtype: float64
Roczna produkcja energii (sumy roczne):
date
1979    2361.20
1980    2344.50
1981    2314.10
1982    2568.20
1983    2496.70
1984    2396.70
1985    2411.00
1986    2555.30
1987    2473.94
1988    2388.50
1989    2573.94
1990    2564.20
1991    2494.80
1992    2630.44
1993    2511.70
1994    2504.14
1995    2509.30
1996    2460.90
1997    2534.90
1998    2401.60
1999    2520.64
2000    2333.78
2001    2283.90
2002    2327.30
2003    2447.70
2004    2327.40
2005    2683.80
2006    2637.14
2007    2649.30
2008    2330.82
2009    2710.60
2010    2676.28
2011    2777.74
2012    2792.34
2013    2754.54
2014    2853.64
2015    2821.80
2016    2736.14
2017    2752.74
2018    2990.34
2019    2869.00
2020    2906.30


Miesięczna produkcja energii jest prezentowana jako suma wszystkich lat dla każdego konkretnego miesiąca. Oznacza to, że wartości, które widzimy, to agregaty miesięczne produkcji energii z wielu lat, które w przypadku stycznia wynoszą na przykład 8719.62 kWh. To sugeruje, że dla każdego stycznia z lat 1979-2023, suma energii wynosi 8719.62 kWh.
Roczna produkcja energii jest prezentowana jako suma produkcji energii dla każdego roku z osobna. Dla przykładu w 1979 roku suma ta wynosi 2361.20 kWh, co jest znacznie niższą wartością w porównaniu do sumy miesięcznej, ponieważ są to dane tylko z jednego roku.
Co to oznacza?
Miesięczne wykresy są akumulacją danych z wielu lat dla danego miesiąca, co daje wysokie wartości sum.
Roczne wykresy pokazują produkcję na przestrzeni jednego roku, co jest bardziej reprezentatywne dla tego, ile energii zostało wyprodukowane w każdym konkretnym roku.
Wnioski:
Porównywanie tych dwóch wykresów bezpośrednio może być mylące, ponieważ reprezentują one dane w różny sposób.
Aby uzyskać bardziej porównywalne dane, można by obliczyć średnią miesięczną produkcję energii na rok, dzieląc miesięczne sumy przez liczbę lat w analizie. Na przykład, suma produkcji w styczniu 8719.62 kWh mogłaby być podzielona przez liczbę lat (np. 45 lat od 1979 do 2023), aby uzyskać średnią miesięczną produkcję dla stycznia każdego roku.
Alternatywnie, jeśli chcesz pokazać roczną sumę w sposób, który jest porównywalny z miesięcznymi danymi, możesz zsumować wszystkie miesięczne produkcje w ramach jednego roku i porównać to z sumą dla każdego roku.
Te rozważania mogą pomóc w lepszym zrozumieniu dynamiki produkcji energii w analizowanym okresie oraz w precyzyjniejszym prezentowaniu danych na wykresach.

trzeba poprawic te wykresy, tak samo jak w 03_ notebooku bo to nie ma sensu.

to jest dla instalcji 5kWh

to z notebooku 03_ - tutaj ustawiamy moc

# Stałe
installed_power_kWp = 5  # Moc zainstalowana w kWp
performance_ratio = 0.80  # Współczynnik wydajności (PR)

# Obliczanie dziennej produkcji energii
df_weather_db['daily_energy_production_kWh'] = (
    installed_power_kWp * 
    df_weather_db['estimated_irradiance'] * 
    performance_ratio * 
    df_weather_db['temp_modifier'] * 
    df_weather_db['humidity_modifier']
)

Szacowane zapotrzebowanie:
1. Wyznaczenie podstawowych wartości zużycia energii dla pomp ciepła:
Pompy ciepła są zazwyczaj bardziej efektywne niż tradycyjne systemy ogrzewania, więc ich zużycie energii jest niższe. Przyjmując, że pompa ciepła ma średnią efektywność i biorąc pod uwagę różne rozmiary domów, można założyć następujące wartości:
-- 70 m²: Około 50 kWh/m² rocznie
-- 100 m²: Około 50 kWh/m² rocznie
-- 150 m²: Około 50 kWh/m² rocznie
2. Kalkulacja rocznego zapotrzebowania na energię:
Dla rodziny z dwojgiem dzieci:
-- 70 m²: 70m2×50kWh/m2=3500kWh70m2×50kWh/m2=3500kWh
-- 100 m²: 100m2×50kWh/m2=5000kWh100m2×50kWh/m2=5000kWh
-- 150 m²: 150m2×50kWh/m2=7500kWh150m2×50kWh/m2=7500kWh
Dla pary (założenie: mniejsze zapotrzebowanie na ciepłą wodę i ogólnie energię):
-- 70 m²: 70m2×50kWh/m2×0.85=2975kWh70m2×50kWh/m2×0.85=2975kWh(15% mniej)
-- 100 m²: 100m2×50kWh/m2×0.85=4250kWh100m2×50kWh/m2×0.85=4250kWh
-- 150 m²: 150m2×50kWh/m2×0.85=6375kWh150m2×50kWh/m2×0.85=6375kWh
3. Uwagi:
    -- Przyjęto, że zapotrzebowanie na energię jest niższe o 15% dla pary w porównaniu z rodziną z dwojgiem dzieci ze względu na mniejsze zużycie ciepłej wody i mniejszą liczbę urządzeń elektrycznych w użytku.
    -- Rzeczywiste zużycie energii może się różnić w zależności od jakości izolacji domu, lokalizacji geograficznej (klimatu), stylu życia mieszkańców oraz wieku i stanu technicznego pompy ciepła.
Wnioski:
Te przybliżone obliczenia dają ogólny obraz potencjalnego zapotrzebowania na energię w domach ogrzewanych za pomocą pompy ciepła. Warto jednak zauważyć, że indywidualne warunki mogą znacząco wpłynąć na rzeczywiste zużycie energii. Dla dokładniejszej analizy i optymalizacji zużycia energii zalecane jest przeprowadzenie audytu energetycznego domu.

Aus <https://chatgpt.com/c/f54165fc-a2e6-4591-987f-8b4ee7be6303> 


In [16]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px


app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Analiza Produkcji Energii"),
    dcc.Dropdown(
        id='city-dropdown',
        options=[{'label': city, 'value': city} for city in df_weather_final['city_name'].unique()],
        value='Baden-Baden',  # Domyślna wartość
        clearable=False
    ),
    dcc.DatePickerRange(
        id='date-picker-range',
        start_date=df_weather_final['date'].min(),
        end_date=df_weather_final['date'].max(),
        display_format='YYYY-MM-DD'
    ),
    dcc.Graph(id='time-series-chart'),
])

@app.callback(
    Output('time-series-chart', 'figure'),
    [Input('city-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_graph(selected_city, start_date, end_date):
    filtered_df = df_weather_final[(df_weather_final['city_name'] == selected_city) & 
                     (df_weather_final['date'] >= start_date) & 
                     (df_weather_final['date'] <= end_date)]
    fig = px.line(filtered_df, x='date', y='daily_energy_production_kWh', title='Daily Energy Production Over Time')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


### loakalizacja automatycznie ma byc brana!!!!!

In [12]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_leaflet as dl

# Dane lokalizacji 
locations = {
    'Baden_Baden': {'lat': 48.765640, 'lon': 8.228524},
    'Bozanska': {'lat': 54.473028, 'lon': 18.302179},
    'Walcz_Drugi': {'lat': 53.257039, 'lon': 16.521203}
}

# Inicjalizacja aplikacji Dash
app = dash.Dash(__name__)

# Struktura aplikacji
app.layout = html.Div([
    dl.Map([
        dl.TileLayer(),  # Warstwa kafelków mapy
        dl.Marker(position=(48.765640, 8.228524), id='map-marker')  # Marker z początkową pozycją
    ], id='map', style={'width': '100%', 'height': '50vh'}, center=(50, 10), zoom=4),
    html.Div([  # Kontener dla menu rozwijanego
        dcc.Dropdown(
            id='location-dropdown',
            options=[{'label': loc, 'value': loc} for loc in locations.keys()],
            value='Bozanska',  # Wartość domyślna
            style={'width': '40%'}  # Zmniejszenie szerokości dropdown
        )
    ], style={'position': 'absolute', 'bottom': 10, 'left': 10, 'width': '100%', 'z-index': '1000'})
], style={'position': 'relative'})

# Callback do aktualizacji markera i centrum mapy
@app.callback(
    [Output('map-marker', 'position'),
     Output('map', 'center')],
    [Input('location-dropdown', 'value')]
)
def update_map(selected_location):
    lat_lon = locations[selected_location]
    return (lat_lon['lat'], lat_lon['lon']), (lat_lon['lat'], lat_lon['lon'])

# Uruchomienie serwera
if __name__ == '__main__':
    app.run_server(debug=True)
